In [19]:
import pandas as pd
import numpy as np
import geopandas as gpd
import geojsonio
import io
import folium
import pickle
import matplotlib.pyplot as plt 
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
print('libraries imported')

libraries imported


# Downloading raw data of RE prices from open data gouv

In [2]:
raw_data = pd.read_csv('https://cadastre.data.gouv.fr/data/etalab-dvf/latest/csv/2018/full.csv.gz', compression='gzip',header=0, sep=',', quotechar='"')
print('Data downloaded')

C:\Users\amaur\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (8,10,12,14,16,17,18,20,22,26,35,36) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


Data downloaded


# Data cleaning

In [20]:
## Know column names and size of the dataset
print(raw_data.columns.tolist())
print(len(raw_data))

['id_mutation', 'date_mutation', 'numero_disposition', 'nature_mutation', 'valeur_fonciere', 'adresse_numero', 'adresse_suffixe', 'adresse_nom_voie', 'adresse_code_voie', 'code_postal', 'code_commune', 'nom_commune', 'code_departement', 'ancien_code_commune', 'ancien_nom_commune', 'id_parcelle', 'ancien_id_parcelle', 'numero_volume', 'lot1_numero', 'lot1_surface_carrez', 'lot2_numero', 'lot2_surface_carrez', 'lot3_numero', 'lot3_surface_carrez', 'lot4_numero', 'lot4_surface_carrez', 'lot5_numero', 'lot5_surface_carrez', 'nombre_lots', 'code_type_local', 'type_local', 'surface_reelle_bati', 'nombre_pieces_principales', 'code_nature_culture', 'nature_culture', 'code_nature_culture_speciale', 'nature_culture_speciale', 'surface_terrain', 'longitude', 'latitude']
3277682


## Selecting relevant variable for analysis, puting variables to the right format, droping NA's ...

In [4]:
# Selecting variables
clean_raw = raw_data[['valeur_fonciere','nom_commune','code_postal','id_parcelle', 'nombre_pieces_principales', 'surface_terrain','type_local', 'surface_reelle_bati','longitude','latitude']]
len(clean_raw)

# Droping NA's values
clean_raw.dropna(subset=['valeur_fonciere','code_postal','nombre_pieces_principales','latitude','longitude'], inplace=True)

# Puting relevant variables from float to integer
cols=['valeur_fonciere','code_postal','nombre_pieces_principales']
clean_raw[cols]=clean_raw[cols].applymap(np.int64)

# Drop duplicate data
clean_raw.drop_duplicates(subset=['id_parcelle'], keep = False, inplace = True)

C:\Users\amaur\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\amaur\Anaconda3\lib\site-packages\pandas\core\frame.py:2963: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]
C:\Users\amaur\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]

## Keep only the data for Toulouse city

In [5]:
toulouse_data = clean_raw[clean_raw['nom_commune'].str.contains("Toulouse")]
toulouse_clean = toulouse_data[toulouse_data.nom_commune!='Vieille-Toulouse']
toulouse = toulouse_clean[toulouse_clean.nom_commune!='Toulouse-le-Château']

### Verification that only Toulouse was kept

In [6]:
toulouse.groupby(['nom_commune']).count()

,valeur_fonciere,code_postal,id_parcelle,nombre_pieces_principales,surface_terrain,type_local,surface_reelle_bati,longitude,latitude
nom_commune,,,,,,,,,
Toulouse,2123,2123,2123,2123,1149,2123,2003,2123,2123


In [7]:
# Reseting indexes for further modification on the database would be feasible
toulouse.reset_index(inplace=True)
toulouse.drop(['index'], axis=1, inplace=True)

C:\Users\amaur\Anaconda3\lib\site-packages\pandas\core\frame.py:3997: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


# Verifying which points (lat-long) belongs to which neighbourhood

In [8]:
# Import packages
import shapely.speedups
from shapely.geometry import Point, Polygon
print('packages loaded')

packages loaded


In [21]:
# Import data
neigbourhood_data = gpd.read_file('C:/Users/amaur/OneDrive/Documents/Data science/MOOC/MOOC coursera/capstone/valeurs foncières/recensement-population-2016-grands-quartiers-logement.geojson')
print('data loaded')

data loaded


In [10]:
# Check the variable
print(neigbourhood_data.columns.tolist())

['p16_rp_4p', 'p16_rp', 'p16_rp_120m2p', 'p16_log', 'p16_pmen_anem10p', 'p16_rpappart_achtot', 'p16_rp_m30m2', 'p16_anem_rp_prop', 'p16_men', 'p16_rp_6080m2', 'p16_rp_ach45', 'p16_rp_3040m2', 'p16_appart', 'p16_rp_100120m2', 'p16_nbpi_rp_anem0509', 'p16_pmen_anem0204', 'p16_rpmaison_ach05', 'p16_rp_grat', 'p16_rpappart_ach45', 'p16_men_anem0509', 'p16_anem_rp', 'p16_rp_4060m2', 'p16_anem_rp_lochlmv', 'p16_rp_ach70', 'p16_rp_2p', 'p16_rp_voit2p', 'grd_quart', 'p16_nper_rp_lochlmv', 'p16_rpmaison_ach13', 'p16_rpappart_ach70', 'p16_rpmaison_ach19', 'p16_maison', 'p16_rp_voit1', 'p16_rp_80100m2', 'p16_rp_3p', 'p16_men_anem0002', 'p16_nbpi_rpappart', 'p16_nbpi_rpmaison', 'uu2010', 'p16_rp_garl', 'p16_rp_ach90', 'p16_rp_prop', 'p16_pmen_anem0509', 'p16_rp_ccind', 'p16_rpmaison_ach90', 'p16_rp_5pp', 'p16_rp_ach19', 'p16_rp_ach13', 'p16_rpmaison_ach70', 'p16_rp_loc', 'p16_rpappart_ach90', 'p16_nbpi_rp_anem0002', 'code_insee', 'p16_rp_achtot', 'p16_rp_sdb', 'p16_anem_rp_grat', 'p16_nbpi_rp_anem

In [23]:
# Select only the geometry and neiborhood name
neigbourhood_data_clean = neigbourhood_data[['libelle_du_grand_quartier','geometry']]
toulouse.head()

,valeur_fonciere,nom_commune,code_postal,id_parcelle,nombre_pieces_principales,surface_terrain,type_local,surface_reelle_bati,longitude,latitude,quartier,prix_m2
0,73000,Toulouse,31000,31555818AB0043,1,NaN,Appartement,17.0,1.438402,43.602411,CAPITOLE,4294.117647
1,140000,Toulouse,31200,31555831BC0687,4,225.0,Maison,77.0,1.446339,43.644810,CROIX-DAURADE,1818.181818
2,76000,Toulouse,31400,31555813AD0294,0,NaN,Appartement,12.0,1.445320,43.586255,SAINT-MICHEL,6333.333333
3,90000,Toulouse,31000,31555818AB0732,2,NaN,Appartement,30.0,1.440546,43.601473,CAPITOLE,3000.000000
4,350057,Toulouse,31000,31555827AD0187,3,90.0,Maison,59.0,1.425701,43.607076,AMIDONNIERS,5933.169492


## Store in list every data point of the toulouse data set

In [24]:
i=0
p=[]
while i<len(toulouse):
    point = Point(toulouse['longitude'][i],toulouse['latitude'][i])
    p.append(point)
    i=i+1

### For every point (shapely format) in p, check if the point is in a given neighbourhood, store this neigbourhood in a list

In [13]:
neigb_list=[]
for points in p:
    for idx, neigb in enumerate(neigbourhood_data_clean['geometry']):
        if points.within(neigb)==True:
            i=idx
            neigb_list.append(neigbourhood_data_clean['libelle_du_grand_quartier'][i])

In [14]:
# Add this column to the toulouse dataset, so we know to which neighborhood belongs which point
toulouse['quartier'] = neigb_list

C:\Users\amaur\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [15]:
# Generate the dependant variable the price per m2 (allows to control price from size)
toulouse['prix_m2'] = toulouse['valeur_fonciere']/toulouse['surface_reelle_bati']

C:\Users\amaur\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [16]:
# Verification of the dataset
toulouse.head(2)

,valeur_fonciere,nom_commune,code_postal,id_parcelle,nombre_pieces_principales,surface_terrain,type_local,surface_reelle_bati,longitude,latitude,quartier,prix_m2
0,73000,Toulouse,31000,31555818AB0043,1,NaN,Appartement,17.0,1.438402,43.602411,CAPITOLE,4294.117647
1,140000,Toulouse,31200,31555831BC0687,4,225.0,Maison,77.0,1.446339,43.644810,CROIX-DAURADE,1818.181818


In [17]:
# Store the dataset for further using
toulouse.to_csv('C:/Users/amaur/OneDrive/Documents/Data science/MOOC/MOOC coursera/capstone/data_toulouse_foncier.csv')

In [18]:
toulouse_map = folium.Map(location=[43.6043, 1.4437], zoom_start=12, titles='Mapbox Bright')

pop_up_insight = 'valeur_fonciere: {}, nombre_pieces_principales: {}'.format(toulouse['valeur_fonciere'], toulouse['nombre_pieces_principales'])
locations = toulouse[['latitude', 'longitude']]
locationlist = locations.values.tolist()
for point in range(0, len(locationlist)):
    folium.Marker(locationlist[point], popup=toulouse['valeur_fonciere'][point]).add_to(toulouse_map)

In [28]:
toulouse_map